In [0]:
!pip install -U -q PyDrive
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os, re, csv, logging, string
import pandas as pd
import seaborn as sns
import random

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# read data from a csv file    
def read_csv (filename, logout = True):
	try:
		reader = csv.reader(open(filename, "r"))
		data = []
		for r in reader:
			data.append(r)
		return data
	except Exception as e:
		if logout is True:
			logging.error(e)
		return None
  
# write data in format of [(x1,y1,z1),(x2,y2,z2)] to a csv file
def write_csv (filename, data, logout = True):
	try:
		doc = csv.writer (open(filename, 'w'), delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
		for d in data:
			doc.writerow (d)
		return True			
	except Exception as e:
		if logout is True:
			log.error(e)
		return False

# remove punctuation
def trans(s):
  exclude = string.punctuation
  return s.translate(str.maketrans({key: None for key in exclude}))

# chunk list to approximatly equal parts
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

# calculate cosine similarity in 10*10 2D array
def tfidf_cosine_sim(model):
    l = []
    sim = []
    for x in range(model.shape[0]):
        l.append(x)
    for m in (l[:10]):
        for n in (l[10:]):
            sim.append(cosine_similarity(model[m], model[n])[0][0])
    return sim

# rank cosine similarity
def rank_question_similarities(model, corpus, corpus_i):
    cos = []
    for x in range(model.shape[0]):
        sim = cosine_similarity(model[0], model[x])
        cos.append([corpus_i[x], corpus[x], sim[0][0]])
    return cos

# calculate cosine similarity under Universal Sentence Encoder
def get_UniSentEnc_sim(corpus, corpus_i):
  module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
  embed = hub.Module(module_url)
  init = tf.global_variables_initializer()
  table_init = tf.tables_initializer()
  with tf.Session() as sess:
    sim = []
    sess.run([init, table_init])
    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    encoding_tensor = embed(similarity_input_placeholder)
    for f in range(len(corpus)):
      en_embeddings_1 = sess.run(encoding_tensor, feed_dict={similarity_input_placeholder: [corpus[0]]})
      en_embeddings_2 = sess.run(encoding_tensor, feed_dict={similarity_input_placeholder: [corpus[f]]})
      sim.append([corpus_i[f], corpus[f], float(cosine_similarity(en_embeddings_1, en_embeddings_2))])
  return sim


# calculate cosine similarity of ELmo
def get_elmo_sim(corpus, corpus_i):
  elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
  init = tf.global_variables_initializer()
  table_init = tf.tables_initializer()
  with tf.Session() as sess:
    sim = []
    sess.run([init, table_init])
    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    encoding_tensor = elmo(similarity_input_placeholder)
    for f in range(len(corpus)):
      en_embeddings_1 = sess.run(encoding_tensor, feed_dict={similarity_input_placeholder: [corpus[0]]})
      en_embeddings_2 = sess.run(encoding_tensor, feed_dict={similarity_input_placeholder: [corpus[f]]})
      sim.append([corpus_i[f], corpus[f], float(cosine_similarity(en_embeddings_1, en_embeddings_2))])
  return sim

In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# load data
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine_lab.csv')
creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q1.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q2.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q3.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q4.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q5.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q6.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q7.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q8.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q9.csv')
#creatinine_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/creatinine/creatinine_Q10.csv')

#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c_lab.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q1.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q2.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q3.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q4.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q5.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q6.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q7.csv')
hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q8.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q9.csv')
#hba1c_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/hba1c/hba1c_Q10.csv')

#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose_lab.csv')
glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q1.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q2.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q3.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q4.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q5.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q6.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q7.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q8.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q9.csv')
#glucose_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/glucose/glucose_Q10.csv')

#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q1.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q2.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q3.csv')
noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q4.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q5.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q6.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q7.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q8.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q9.csv')
#noLab_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/noLab/noLab_Q10.csv')

creatinine_id = [d[0] for d in creatinine_d]
hba1c_id = [d[0] for d in hba1c_d]
glucose_id = [d[0] for d in glucose_d]
noLab_id = [d[0] for d in noLab_d]

creatinine = [d[1].lower() for d in creatinine_d]
hba1c = [d[1].lower() for d in hba1c_d]
glucose = [d[1].lower() for d in glucose_d]
noLab = [d[1].lower() for d in noLab_d]

#creatinine_exp = [d[4] for d in creatinine_d]
#hba1c_exp = [d[4] for d in hba1c_d]
#glucose_exp = [d[4] for d in glucose_d]

# remove punctuation
creatinine_np = [trans(c) for c in creatinine]
hba1c_np = [trans(c) for c in hba1c]
glucose_np = [trans(c) for c in glucose]
noLab_np = [trans(c) for c in noLab]

# tokenization
creatinine_token = [word_tokenize(c) for c in creatinine_np]
hba1c_token = [word_tokenize(c) for c in hba1c_np]
glucose_token = [word_tokenize(c) for c in glucose_np]
noLab_token = [word_tokenize(c) for c in noLab_np]

# remove stopwords
words_stop = [str(c) for c in stopwords.words('english')]
creatinine_no_stopw = [[cc for cc in c if cc not in words_stop]for c in creatinine_token]
hba1c_no_stopw = [[cc for cc in c if cc not in words_stop]for c in hba1c_token]
glucose_no_stopw = [[cc for cc in c if cc not in words_stop]for c in glucose_token]
noLab_no_stopw = [[cc for cc in c if cc not in words_stop]for c in noLab_token]

# stemming words
ps = nltk.stem.PorterStemmer()
creatinine_stemw = [[str(ps.stem(s)) for s in sw] for sw in creatinine_no_stopw]
hba1c_stemw = [[str(ps.stem(s)) for s in sw] for sw in hba1c_no_stopw]
glucose_stemw = [[str(ps.stem(s)) for s in sw] for sw in glucose_no_stopw]
noLab_stemw = [[str(ps.stem(s)) for s in sw] for sw in noLab_no_stopw]

# convert tokens back to sentence
creatinine_stemw_s = [' '.join(i) for i in creatinine_stemw]
hba1c_stemw_s = [' '.join(i) for i in hba1c_stemw]
glucose_stemw_s = [' '.join(i) for i in glucose_stemw]
noLab_stemw_s = [' '.join(i) for i in noLab_stemw]

# vectorize using tfidf
vectorizer = TfidfVectorizer(min_df=1)
C = vectorizer.fit_transform(creatinine_stemw_s)
H = vectorizer.fit_transform(hba1c_stemw_s)
G = vectorizer.fit_transform(glucose_stemw_s)
N = vectorizer.fit_transform(noLab_stemw_s)

#out = rank_question_similarities(C, creatinine, creatinine_id)
#out = rank_question_similarities(H, hba1c, hba1c_id)
#out = rank_question_similarities(G,glucose, glucose_id)
#out = rank_question_similarities(N,noLab, noLab_id)
#write_csv('/content/drive/My Drive/Thesis_feature_sim/test.csv', out)

True

In [0]:
# Universal Sentence Encoder: calculate cosine similarity
#sim = get_UniSentEnc_sim(creatinine, creatinine_id)
#sim = get_UniSentEnc_sim(hba1c, hba1c_id)
#sim = get_UniSentEnc_sim(glucose, glucose_id)
#sim = get_UniSentEnc_sim(noLab, noLab_id)

# ELMo: caculate cosine similarity
#sim = get_elmo_sim(creatinine, creatinine_id)
sim = get_elmo_sim(hba1c, hba1c_id)
#sim = get_elmo_sim(glucose, glucose_id)
#sim = get_elmo_sim(noLab, noLab_id)

write_csv('/content/drive/My Drive/Thesis_feature_sim/test.csv', sim)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


True

In [0]:
# load mixed data
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q1.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q2.csv')
hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q3.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q4.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q5.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q6.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q7.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q8.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q9.csv')
#hba1c_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/HbA1c_top10mix_Q10.csv')

#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q1.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q2.csv')
noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q3.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q4.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q5.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q6.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q7.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q8.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q9.csv')
#noLab_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/NoLab_top10mix_Q10.csv')

#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q1.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q2.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q3.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q4.csv')
glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q5.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q6.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q7.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q8.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q9.csv')
#glucose_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Glucose_top10mix_Q10.csv')

#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q1.csv')
creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q2.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q3.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q4.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q5.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q6.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q7.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q8.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q9.csv')
#creatinine_mix_d = read_csv('/content/drive/My Drive/Thesis_feature_sim/mix_csv/Creatinine_top10mix_Q10.csv')

# extract id
hba1c_mix_id = [d[0] for d in hba1c_mix_d]
noLab_mix_id = [d[0] for d in noLab_mix_d]
glucose_mix_id = [d[0] for d in glucose_mix_d]
creatinine_mix_id = [d[0] for d in creatinine_mix_d]

# extract question and convert to lowercase
hba1c_mix = [d[1].lower() for d in hba1c_mix_d]
noLab_mix = [d[1].lower() for d in noLab_mix_d]
glucose_mix = [d[1].lower() for d in glucose_mix_d]
creatinine_mix = [d[1].lower() for d in creatinine_mix_d]

# remove punctuation
hba1c_mix_np = [trans(c) for c in hba1c_mix]
noLab_mix_np = [trans(c) for c in noLab_mix]
glucose_mix_np = [trans(c) for c in glucose_mix]
creatinine_mix_np = [trans(c) for c in creatinine_mix]

# tokenization
hba1c_mix_token = [word_tokenize(c) for c in hba1c_mix_np]
noLab_mix_token = [word_tokenize(c) for c in noLab_mix_np]
glucose_mix_token = [word_tokenize(c) for c in glucose_mix_np]
creatinine_mix_token = [word_tokenize(c) for c in creatinine_mix_np]

# remove stopwords
words_stop = [str(c) for c in stopwords.words('english')]
hba1c_mix_no_stopw = [[cc for cc in c if cc not in words_stop]for c in hba1c_mix_token]
noLab_mix_no_stopw = [[cc for cc in c if cc not in words_stop]for c in noLab_mix_token]
glucose_mix_no_stopw = [[cc for cc in c if cc not in words_stop]for c in glucose_mix_token]
creatinine_mix_no_stopw = [[cc for cc in c if cc not in words_stop]for c in creatinine_mix_token]

# stemming words
ps = nltk.stem.PorterStemmer()
hba1c_mix_stemw = [[str(ps.stem(s)) for s in sw] for sw in hba1c_mix_no_stopw]
noLab_mix_stemw = [[str(ps.stem(s)) for s in sw] for sw in noLab_mix_no_stopw]
glucose_mix_stemw = [[str(ps.stem(s)) for s in sw] for sw in glucose_mix_no_stopw]
creatinine_mix_stemw = [[str(ps.stem(s)) for s in sw] for sw in creatinine_mix_no_stopw]

# convert tokens back to sentence
hba1c_mix_stemw_s = [' '.join(i) for i in hba1c_mix_stemw]
noLab_mix_stemw_s = [' '.join(i) for i in noLab_mix_stemw]
glucose_mix_stemw_s = [' '.join(i) for i in glucose_mix_stemw]
creatinine_mix_stemw_s = [' '.join(i) for i in creatinine_mix_stemw]

# vectorize using tfidf
vectorizer = TfidfVectorizer(min_df=1)
Hm = vectorizer.fit_transform(hba1c_mix_stemw_s)
Nm = vectorizer.fit_transform(noLab_mix_stemw_s)
Gm = vectorizer.fit_transform(glucose_mix_stemw_s)
Cm = vectorizer.fit_transform(creatinine_mix_stemw_s)

#out = rank_question_similarities(Hm, hba1c_mix, hba1c_mix_id)
#out = rank_question_similarities(Nm,noLab_mix, noLab_mix_id)
out = rank_question_similarities(Gm,glucose_mix, glucose_mix_id)
#out = rank_question_similarities(Cm,creatinine_mix, creatinine_mix_id)
write_csv('/content/drive/My Drive/Thesis_feature_sim/test.csv', out)

True

In [0]:
# Universal Sentence Encoder: calculate cosine similarity
#sim = get_UniSentEnc_sim(hba1c_mix, hba1c_mix_id)
#sim = get_UniSentEnc_sim(noLab_mix, noLab_mix_id)
#sim = get_UniSentEnc_sim(glucose_mix, glucose_mix_id)
#sim = get_UniSentEnc_sim(creatinine_mix, creatinine_mix_id)

# ELMo: caculate cosine similarity
sim = get_elmo_sim(hba1c_mix, hba1c_mix_id)
#sim = get_elmo_sim(noLab_mix, noLab_mix_id)
#sim = get_elmo_sim(glucose_mix, glucose_mix_id)
#sim = get_elmo_sim(creatinine_mix, creatinine_mix_id)

write_csv('/content/drive/My Drive/Thesis_feature_sim/test.csv', sim)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


True